# Anomaly Detection With Clustering

Many data sets take the form of vector values.
One relatively simple but very effective way of defining expected behavior for this kind of data is to cluster it, and use these clusters as a model of our expectations.
Data samples that do not fall inside, or near, any cluster are often anomalous in some way.

In this notebook we will work with
[Amazon Fine Food Reviews](https://www.kaggle.com/snap/amazon-fine-food-reviews/)
data[1] from Kaggle.
We will generate feature vectors from this data and _cluster_ them to use as an anomaly detection model.

[1] J. McAuley and J. Leskovec. From amateurs to connoisseurs: modeling the evolution of user expertise through online reviews. WWW, 2013.

Parquet files used in this notebook were created from the raw Kaggle CSV as follows:
```python
with open("data/amazon-reviews.csv") as f:
    data = pd.read_csv(f)
data = data.sample(10000).reset_index(drop=True)
data = data.drop(columns=["Id", "ProductId", "UserId", "ProfileName", "Time", "Summary"])
data["hscore"] = \
    data.apply(lambda row: (1+row["HelpfulnessNumerator"]) / (2+row["HelpfulnessDenominator"]), axis=1)
data = data.drop(columns=["HelpfulnessNumerator", "HelpfulnessDenominator"])
data = data.rename(columns={"Score":"score", "Text":"text"})
data = data[["score", "hscore", "text"]]
data.to_parquet("data/amazon-reviews-10K.parquet", compression="brotli")
```

In [ ]:
#!pip install pyarrow

In [ ]:
import codecs
import random
import math
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import re

# Loading the data

The raw food review data has been sub-sampled to 50,000 records and stored as a parquet file to reduce its footprint on disk.

We begin by loading the data.
You can see that each review comes with a score, from one to five "stars", a helpfulness score, and the review text.
In this lab we will not be using the helpfulness score.

In [ ]:
reviews = pd.read_parquet("data/amazon-reviews-50K.parquet").reindex()

In [ ]:
htmlbr = re.compile('<br />')
whitesp = re.compile('\\s+')
def cleantxt(txt):
    clean = re.sub(htmlbr, ' ', txt)
    clean = re.sub(whitesp, ' ', clean)
    clean = clean.lower()
    return clean

# Anomalies with Word Features

The previous two variations on feature vectors for anomaly detection suggest that there is no one way to define what is "anomalous".
What we detect as an anomaly depends on how we define our expectations.
That in turn depends on what kind of features we collect in the first place.

With that in mind, what will happen if we replace shingles with whole words for generating hashed frequency vectors?

In the following cells we apply the SKLearn hashing vectorizer to create a hashed vector of word counts.
As before, we will normalize these vectors to a length of 1 to put different review lengths on an equal footing.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

HVSIZE = 100
vectorizer = HashingVectorizer(token_pattern='(?u)\\b[A-Za-z]\\w+\\b', n_features = HVSIZE, alternate_sign=False)
hvcounts = vectorizer.fit_transform(reviews["text"].apply(cleantxt))

In [ ]:
def normarray(v):
    r = v.toarray().reshape(HVSIZE)
    z = np.linalg.norm(r)
    if (z > 0.0): r /= z
    return r

reviews["feats"] = [normarray(v) for v in hvcounts]

In [ ]:
data = np.array(list(reviews["feats"]))
clustering = KMeans(n_clusters=10).fit(data)

In [ ]:
requirements = [["scikit-learn", "0.21.3"], ["pandas", "0.23.0"], ["numpy", "1.17.0"], ["scipy", "1.3.1"], ["pyarrow", "0.15.1"]]

In [ ]:
def predictor(x):
    xclean = cleantxt(x)
    xvec = normarray(vectorizer.transform([xclean]))
    xj = clustering.predict([xvec])[0]
    xc = clustering.cluster_centers_[xj]
    return np.linalg.norm(xc - xvec)

In [ ]:
def validator(x):
    return isinstance(x, str)

In [ ]:
#%pip freeze

In [ ]:
#x = reviews["text"][0]
#predictor(x)